In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
import pandas as pd 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import numpy as np
import random
from rdkit import Chem
from keras.layers.merge import concatenate

In [122]:
df = pd.read_csv('drive/MyDrive/fda.csv')

Let us dig deeper on the length of the SMILES to figure out ideal embedding length

In [123]:
df['length'] = df['smiles'].apply(len)

In [124]:
df.length.describe()

count    1615.000000
mean       53.447059
std        31.727066
min         4.000000
25%        33.000000
50%        47.000000
75%        65.000000
max       210.000000
Name: length, dtype: float64

With 75 percentile about 65, we have picked the embedding length to be 100 X len(vocabulary)

In [125]:
MAX_LEN = 100
def get_normalized_text(text):
    text_normalisation = []
    for i in range(MAX_LEN):
        if i >= len(text):
            text_normalisation.append('e')
        else:
            text_normalisation.append(text[i])
    return ''.join(text_normalisation)
df['normalized_smiles'] = df['smiles'].apply(lambda x: get_normalized_text(x))

In [126]:
def get_vocab_dict(data):
    t = Tokenizer(char_level=True)
    t.fit_on_texts(data)
    return t.word_index, t.index_word

In [127]:
def one_hot_encoding(text,vocab_dict):
    text = text.lower()
    vector = np.zeros([MAX_LEN, len(vocab_dict)])
    for i,letter in enumerate(text):
        if i > MAX_LEN:
            break
        vector[i][vocab_dict[letter]-1] = 1
    return vector.reshape(MAX_LEN,len(vocab_dict),1)

In [128]:
vocab_dict,reverse_vocab_dict = get_vocab_dict(df['normalized_smiles'].tolist())


In [129]:
training_data= []
for i,j in df.iterrows():
    training_data.append(one_hot_encoding(j['normalized_smiles'], vocab_dict))
training_data = np.array(training_data)

#### VARIATIONAL AUTO-ENCODERS

In [130]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [131]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

**Encoder**

Using three channel CNN for the encoder so as to capture the spatial relationship and the sequence of the SMILES string. 3 channels with filter size (6*30), (4*30) and (2,30) where 30 is the size of the vocabulary. Convolutions will not move horizontally because unlike images there is no relationship between adjacent columns

In [132]:
latent_dim = 32

encoder_inputs1 = keras.Input(shape=(100, 30,1))

## CHANNEL 1
x1 = layers.Conv2D(5, (6,30), activation="relu", strides=1)(encoder_inputs1)
x1 = layers.Flatten()(x1)

## CHANNEL 2
x2 = layers.Conv2D(5, (4,30), activation="relu", strides=1)(encoder_inputs1)
x2 = layers.Flatten()(x2)

## CHANNEL 3
x3 = layers.Conv2D(5, (2,30), activation="relu", strides=1)(encoder_inputs1)
x3 = layers.Flatten()(x3)

merged = concatenate([x1, x2, x3])
merged = layers.Dense(256, activation="relu")(merged)
merged = layers.Dense(128, activation="relu")(merged)


z_mean = layers.Dense(latent_dim, name="z_mean")(merged)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(merged)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs1, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 100, 30, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 95, 1, 5)     905         input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 97, 1, 5)     605         input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 99, 1, 5)     305         input_25[0][0]                   
____________________________________________________________________________________________

**Decoder**

Used a simple MultiLayered Perceptron for the decoding part.


In [133]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(96, activation="relu")(latent_inputs)
x = layers.Dense(1024, activation="relu")(x)

x = layers.Dense(3000, activation="sigmoid")(x)
decoder_outputs = layers.Reshape((100, 30, 1))(x)
# x = layers.Conv2DTranspose(5, (2,30), activation="relu", strides=1, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, (100,30), activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()


Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_50 (Dense)             (None, 96)                3168      
_________________________________________________________________
dense_51 (Dense)             (None, 1024)              99328     
_________________________________________________________________
dense_52 (Dense)             (None, 3000)              3075000   
_________________________________________________________________
reshape_8 (Reshape)          (None, 100, 30, 1)        0         
Total params: 3,177,496
Trainable params: 3,177,496
Non-trainable params: 0
_________________________________________________________________


In [134]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [135]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())


In [142]:
vae.fit(training_data, epochs=1000, batch_size=128)

Epoch 1/1000
13/13 [==============================] - 0s 4ms/step - loss: 115.4173 - reconstruction_loss: 100.7753 - kl_loss: 14.9509
Epoch 2/1000
13/13 [==============================] - 0s 4ms/step - loss: 113.3580 - reconstruction_loss: 100.1209 - kl_loss: 14.8770
Epoch 3/1000
13/13 [==============================] - 0s 4ms/step - loss: 113.0379 - reconstruction_loss: 99.5743 - kl_loss: 15.0053
Epoch 4/1000
13/13 [==============================] - 0s 4ms/step - loss: 114.4988 - reconstruction_loss: 99.2830 - kl_loss: 15.0699
Epoch 5/1000
13/13 [==============================] - 0s 4ms/step - loss: 112.3362 - reconstruction_loss: 99.2274 - kl_loss: 15.1380
Epoch 6/1000
13/13 [==============================] - 0s 4ms/step - loss: 112.6209 - reconstruction_loss: 97.4644 - kl_loss: 15.3510
Epoch 7/1000
13/13 [==============================] - 0s 4ms/step - loss: 110.0048 - reconstruction_loss: 97.1797 - kl_loss: 15.2153
Epoch 8/1000
13/13 [==============================] - 0s 4ms/step -

Generate molecule through the latent space.

In [143]:
def generate_molecule():
  prediction = decoder.predict([[random.uniform(-1,1) for i in range(32)]])
  string_list = []
  for char in prediction[0]:
      elements = [element[0] for element in char]
      final_char = reverse_vocab_dict.get(1 + elements.index(max(elements)))
      if final_char == 'e':
        break
      string_list.append(final_char)
  return ''.join(string_list)

In [144]:
#Example
molecules = []
for i in range(5):
  m = generate_molecule()
  molecules.append(m)

pd.DataFrame(molecules,columns = ['Molecule'])

,Molecule
0,cc1nc(cnc2co()cncc)cc3[c@o)cc)=o)cc=o)c
1,ccc1c(n2cc([c@(1coccc3cncccc2c(c)c3cc33nho)ccc...
2,cc1cc(cnc(c2)(=ococc(((ccccc(c=c)cc=c2cc2cc1
3,oc1nc@([ccn2oc((co(ccoc[)h]((n)[o@)(])c)(=oc1o...
4,oc1nn((1ccc=oon1ccccc(c=o))(=o)noc()c(o)co)c(@...


To ensure that the SMILES representation which is generated is legitimate, we use the library RDKit to validate. As can be seen, among 10000 simulations we get a few molecules.

In [119]:
for i in range(10000):
  molecule = generate_molecule()
  m = Chem.MolFromSmiles(molecule)
  if m:
    print(molecule)

Exception ignored in: <bound method IteratorResourceDeleter.__del__ of <tensorflow.python.data.ops.iterator_ops.IteratorResourceDeleter object at 0x7f034f3ef448>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 535, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


KeyboardInterrupt: ignored